In [1]:
from IPython.display import display, Image
from PIL import Image as PIL_Image
from io import BytesIO
import numpy as np
from IPython.display import display, Image

def display_img_array(ima, **kwargs):
    if ima.dtype == np.float32 or ima.dtype == np.float64:
        ima = (np.clip(ima, 0., 1.)*255).astype(np.uint8)
    im = PIL_Image.fromarray(ima)
    bio = BytesIO()
    im.save(bio, format='png')
    display(Image(bio.getvalue(), format='png', **kwargs))

IMG_C, IMG_W, IMG_H=1,28,28
def show(x):
    x = x.reshape(-1, IMG_C, IMG_H, IMG_W)
    w = min(x.shape[0]*100, 800)
    x = x.swapaxes(0, 1).swapaxes(1,2)
    if IMG_C == 1:
        img = x.reshape(IMG_H, -1)
    else:
        x = x.reshape(IMG_C, IMG_H, -1)
        img = np.swapaxes(x, 0, 1)
        img = np.swapaxes(img, 1, 2)
    display_img_array(img, width=w) 

In [2]:
from keras.utils import to_categorical
from keras import layers, models, optimizers ,Model
from keras.layers import Input 
def load_mnist():
    # the data, shuffled and split between train and test sets
    from keras.datasets import mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.
    x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.
    y_train = to_categorical(y_train.astype('float32'))
    y_test = to_categorical(y_test.astype('float32'))
    return (x_train, y_train), (x_test, y_test)
(x_train, y_train), (x_test, y_test) = load_mnist()

Using TensorFlow backend.
/home/yiwen/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
from keras import backend as K
from keras.layers import Activation 
from keras.utils.generic_utils import get_custom_objects
def swish(x):
    return K.sigmoid(x)*x
get_custom_objects().update({'swish': Activation(swish )})

In [14]:
D=models.Sequential()
D.add(layers.Conv2D(input_shape=(28, 28, 1),filters=256,kernel_size=(3,3),activation='relu',padding='same'))
D.add(layers.Dropout(0.3))
D.add(layers.MaxPool2D())
D.add(layers.Conv2D(filters=10,kernel_size=(3,3),activation='relu',padding='same'))
D.add(layers.Dropout(0.3))
D.add(layers.Conv2D(filters=10,kernel_size=(14,14),activation='softmax',padding='valid'))
#D.add(layers.Activation('softmax'))
D.add(layers.Flatten())
D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 28, 28, 256)       2560      
_________________________________________________________________
dropout_7 (Dropout)          (None, 28, 28, 256)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 10)        23050     
_________________________________________________________________
dropout_8 (Dropout)          (None, 14, 14, 10)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 1, 1, 10)          19610     
_________________________________________________________________
flatten_1 (Flatten)          (None, 10)                0         
Total para

In [19]:
D.compile(loss='mse',optimizer='Adam',metrics=['accuracy'])
D.fit(x_train,y_train,epochs=10,batch_size=400)
D.evaluate(x_test,y_test)

Epoch 1/10
60000/60000 [==============================] - 20s 327us/step - loss: 0.0024 - acc: 0.9849
Epoch 2/10
60000/60000 [==============================] - 19s 323us/step - loss: 0.0022 - acc: 0.9861
Epoch 3/10
60000/60000 [==============================] - 19s 323us/step - loss: 0.0021 - acc: 0.9868
Epoch 4/10
60000/60000 [==============================] - 19s 323us/step - loss: 0.0020 - acc: 0.9871
Epoch 5/10
60000/60000 [==============================] - 19s 323us/step - loss: 0.0020 - acc: 0.9877
Epoch 6/10
60000/60000 [==============================] - 19s 324us/step - loss: 0.0018 - acc: 0.9881
Epoch 7/10
60000/60000 [==============================] - 19s 322us/step - loss: 0.0018 - acc: 0.9891
Epoch 8/10
60000/60000 [==============================] - 19s 323us/step - loss: 0.0018 - acc: 0.9889
Epoch 9/10
60000/60000 [==============================] - 19s 323us/step - loss: 0.0016 - acc: 0.9898
Epoch 10/10
60000/60000 [==============================] - 19s 323us/step - loss: 

In [34]:
D_train=D.predict(x_train)

In [74]:
Decoder=models.Sequential()
Decoder.add(layers.Dense(input_shape=(10,),units=1960,activation='swish'))
Decoder.add(layers.Dropout(0.3))
Decoder.add(layers.Dense(units=1960,activation='swish'))
Decoder.add(layers.Dropout(0.3))
Decoder.add(layers.Dense(units=1960,activation='swish'))
Decoder.add(layers.Dropout(0.3))
Decoder.add(layers.Dense(units=512,activation='swish'))
Decoder.add(layers.Dropout(0.3))
Decoder.add(layers.Dense(units=784,activation='relu'))
Decoder.add(layers.Reshape((28,28,1)))
Decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 1960)              21560     
_________________________________________________________________
dropout_13 (Dropout)         (None, 1960)              0         
_________________________________________________________________
dense_50 (Dense)             (None, 1960)              3843560   
_________________________________________________________________
dropout_14 (Dropout)         (None, 1960)              0         
_________________________________________________________________
dense_51 (Dense)             (None, 1960)              3843560   
_________________________________________________________________
dropout_15 (Dropout)         (None, 1960)              0         
_________________________________________________________________
dense_52 (Dense)             (None, 512)               1004032   
__________

In [96]:
Decoder.compile(loss='mse',optimizer='sgd')
Decoder.fit(D_train,x_train,epochs=10,batch_size=2000)

Epoch 1/10
60000/60000 [==============================] - 2s 36us/step - loss: 0.0544
Epoch 2/10
60000/60000 [==============================] - 2s 26us/step - loss: 0.0543
Epoch 3/10
60000/60000 [==============================] - 2s 26us/step - loss: 0.0543
Epoch 4/10
60000/60000 [==============================] - 2s 26us/step - loss: 0.0543
Epoch 5/10
60000/60000 [==============================] - 2s 26us/step - loss: 0.0543
Epoch 6/10
60000/60000 [==============================] - 2s 26us/step - loss: 0.0543
Epoch 7/10
60000/60000 [==============================] - 2s 27us/step - loss: 0.0543
Epoch 8/10
60000/60000 [==============================] - 2s 26us/step - loss: 0.0543
Epoch 9/10
60000/60000 [==============================] - 2s 26us/step - loss: 0.0543
Epoch 10/10
60000/60000 [==============================] - 2s 26us/step - loss: 0.0543


In [95]:
print('還原圖')
reco=Decoder.predict(D_train)
show(reco[1:11])
print('Uniform生成')
a = np.random.random((10,10))
reco = Decoder.predict(a)
show(reco)
print('Normal生成')
a = np.random.normal(0,1,(10,10))
reco = Decoder.predict(a)
show(reco)

還原圖


Uniform生成


Normal生成
